In [1]:
from datetime import datetime

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [2]:
events_data_train = pd.read_csv('data/event_data_train.zip')
events_data_train.head()

,step_id,timestamp,action,user_id
0,32815,1434340848,viewed,17632
1,32815,1434340848,passed,17632
2,32815,1434340848,discovered,17632
3,32811,1434340895,discovered,17632
4,32811,1434340895,viewed,17632


In [3]:
submission_data_train = pd.read_csv('data/submissions_data_train.zip')
submission_data_train.head()

,step_id,timestamp,submission_status,user_id
0,31971,1434349275,correct,15853
1,31972,1434348300,correct,15853
2,31972,1478852149,wrong,15853
3,31972,1478852164,correct,15853
4,31976,1434348123,wrong,15853


# preprocessing

In [4]:
def main_features(events, submission):
    # таблица с действиями юзеров
    events_pivot = pd.pivot_table(events, 
                                  values='step_id',
                                  index='user_id', 
                                  columns='action',
                                  aggfunc='count', 
                                  fill_value=0) \
                               .reset_index() \
                               .rename_axis('', axis=1)
    
    # таблица с количеством правильных и неправильных ответов
    submissions_pivot = pd.pivot_table(submission, 
                                       values='step_id',
                                       index='user_id',
                                       columns='submission_status',
                                       aggfunc='count',
                                       fill_value=0) \
                                    .reset_index() \
                                    .rename_axis('', axis=1)
    
    users_data = submissions_pivot.merge(events_pivot, on='user_id', how='outer').fillna(0)
    
    assert users_data['user_id'].nunique() == events['user_id'].nunique()
    
    return users_data

In [5]:
# Количество дней обучения юзера, 
# на основе которых будем предсказывать вероятность того, 
# что юзер пройдет курс до конца
days_count = 2

# Дни в секундах
limit_period = days_count * 24 * 3600

# Порог: если юзер решил correct_answers_count практических заданий,
# то предполагаем, что он закончит курс
# (функция target)
correct_answers_count = 40

In [6]:
def datetime_filter(df):
    # отбираем записи действий юзеров в течение days_count-дней с начала учебы
    
    # вычисляем дату первого действия юзера на курсе
    learning_start_time = df.groupby('user_id').agg({'timestamp': 'min'}) \
                            .rename(columns={'timestamp': 'min_timestamp'}) \
                            .reset_index()
    
    datetime_filtered = df.merge(learning_start_time, how='outer')
    
    # отбираем записи действий юзеров в течение двух дней с начала учебы
    datetime_filtered = datetime_filtered[datetime_filtered['timestamp'] <= datetime_filtered['min_timestamp'] + limit_period]
    
    # проверяем, что в процессе фильтрации не потеряли юзеров
    assert datetime_filtered['user_id'].nunique() == df['user_id'].nunique()
    
    datetime_filtered = datetime_filtered.drop(['min_timestamp'], axis=1)

    return datetime_filtered

In [7]:
def unique_steps_counter(submission):
    # количество уникальных степов, которые юзер пробовал выполнить
    unique_steps_count = submission.groupby('user_id')['step_id'].nunique().to_frame().reset_index() \
                                        .rename(columns={'step_id': 'unique_steps_count'})
    
    return unique_steps_count

In [8]:
def correct_answers(df):
    # получаем долю правильных ответов для каждого юзера
    df['correct'] = (df['correct'] / (df['correct'] + df['wrong'])).fillna(0)

    return df

In [9]:
def target(submission):
    # предполагаем, если пользователь выполнил 40 заданий, то он закончит курс
    
    users_count_correct = submission[submission['submission_status'] == 'correct'] \
                .groupby('user_id').agg({'step_id': 'count'}) \
                .reset_index().rename(columns={'step_id': 'correct_answers_count'})
    
    users_count_correct['passed_course'] = users_count_correct['correct_answers_count'] >= correct_answers_count

    users_count_correct = users_count_correct.drop(['correct_answers_count'], axis=1)
    return users_count_correct

# формируем тренировочный и тестовый датасэты

In [10]:
def create_train_df(events, submission):
    # формируем тренировочный датасэт в признаками и таргетом
    
    # фильтруем данные по дням от начала учебы
    events_2days = datetime_filter(events)
    submissions_2days = datetime_filter(submission)
    
    # создаем таблицу с базовыми фичами
    users_data = main_features(events_2days, submissions_2days)
    
    # создаем целевую переменную
    users_count_correct = target(submission)

    # создаем фичи с попытками степов и долей правильных ответов
    unique_steps_count = unique_steps_counter(submissions_2days)
    users_data = correct_answers(users_data)
    
    # соединяем шаги
    merge1 = users_data.merge(unique_steps_count, how='outer').fillna(0)
    # присоединяем целевую переменную
    merge2 = merge1.merge(users_count_correct, how='outer').fillna(0)
    
    # отделяем целевую переменную и удаляем ее из основного датасета
    y = merge2['passed_course'].map(int)
    X = merge2.drop(['passed_course'], axis=1)
    
    return X, y

In [11]:
def create_test_df(events, submission):
    # формируем тренировочный датасэт
    
    # фильтруем данные по дням от начала учебы
    events_2days = datetime_filter(events)
    submissions_2days = datetime_filter(submission)
    
    # создаем таблицу с базовыми фичами
    users_data = main_features(events_2days, submissions_2days)
    
    # создаем фичи с попытками степов и долей правильных ответов
    unique_steps_count = unique_steps_counter(submissions_2days)
    users_data = correct_answers(users_data)

    X = users_data.merge(unique_steps_count, how='outer').fillna(0)
       
    return X

In [12]:
# тренировочный датасет
events_data_train = pd.read_csv('data/event_data_train.zip')
submission_data_train = pd.read_csv('data/submissions_data_train.zip')

# тестовый датасет
events_data_test = pd.read_csv('data/events_data_test.zip')
submission_data_test = pd.read_csv('data/submission_data_test.zip')

In [13]:
# создание тренировочного датасета
X_train, y_train = create_train_df(events_data_train, submission_data_train)

In [14]:
# создание тестового датасета
X_test = create_test_df(events_data_test, submission_data_test)

# обучение модели

In [15]:
def best_grid(train_data, y):
    # Обучение модели и вычисление лучших параметров модели 
    X_train, X_test, y_train, y_test = train_test_split(train_data, 
                                                        y, 
                                                        test_size=0.3, 
                                                        random_state=40)
    
    parameters = {'n_estimators': range(20, 45, 2), 
                  'max_depth': range(5, 13)}
    
    clf = RandomForestClassifier()
    grid = GridSearchCV(clf, 
                        param_grid=parameters, 
                        cv=5)
    grid.fit(X_train, y_train)
    best_params = grid.best_params_
    print(f'Лучшие параметры модели: {best_params}')
    ypred_prob = grid.predict_proba(X_test)
    roc_score = roc_auc_score(y_test, ypred_prob[:, 1])
    score = grid.score(X_train, y_train)
    print(f'Доля верных предсказаний на тренировочных данных: {score:.3f}')
    print(f'roc_score: {roc_score:.3f}')
    return best_params

In [16]:
start_time = datetime.now()
best_params = best_grid(X_train, y_train)
end_time = datetime.now()
print(f'Время работы: {end_time - start_time}')

Лучшие параметры модели: {'max_depth': 7, 'n_estimators': 42}
Доля верных предсказаний на тренировочных данных: 0.912
roc_score: 0.891
Время работы: 0:01:03.201102


In [17]:
max_depth = best_params['max_depth']
n_estimators = best_params['n_estimators']

In [18]:
def prediction(train_df, y, test_df):
    # Обучение с лучшими параметрами
    test_data = test_df.sort_values('user_id')
    X_train, X_test, y_train, y_test = train_test_split(train_df, 
                                                        y, 
                                                        test_size=0.3, 
                                                        random_state=40)
    best_clf = RandomForestClassifier(max_depth=max_depth,
                                      n_estimators=n_estimators,  
                                      random_state=42)
    best_clf.fit(X_train, y_train)

    ypred_prob = best_clf.predict_proba(X_test)
    
    roc_score = roc_auc_score(y_test, ypred_prob[:, 1])
    score = best_clf.score(X_test, y_test)
    
    print(f'Доля верных предсказаний на тестовых данных: {score:.3f}')
    print(f'roc_score тестовых данных : {roc_score:.3f}')
    
    predictions = best_clf.predict_proba(test_data)
    res = test_data['user_id'].to_frame()
    
    # из массива, сформированного predict_proba, 
    # оставляем только поле, показывающее вероятность, что юзер закончит курс
    res['is_gone'] = predictions[:, 1]
    # запись в csv
    res[['user_id', 'is_gone']].to_csv('predictions.csv', index=False)

In [19]:
prediction(X_train, y_train, X_test)

Доля верных предсказаний на тестовых данных: 0.902
roc_score тестовых данных : 0.891
